In [26]:
from pynq import PL
PL.reset()
from pynq import Overlay  #import the overlay module
ol = Overlay('./spi_tx/spi_tx.bit')  #locate/point to the bit file

In [27]:
import pprint
pprint.pprint(ol.ip_dict)

{'disp_interface_0': {'addr_range': 65536,
                      'bdtype': None,
                      'device': <pynq.pl_server.embedded_device.EmbeddedDevice object at 0xb38f6e08>,
                      'driver': <class 'pynq.overlay.DefaultIP'>,
                      'fullpath': 'disp_interface_0',
                      'gpio': {},
                      'interrupts': {},
                      'mem_id': 'S00_AXI',
                      'memtype': 'REGISTER',
                      'parameters': {'ADDR_WIDTH': '5',
                                     'ARUSER_WIDTH': '0',
                                     'AWUSER_WIDTH': '0',
                                     'BUSER_WIDTH': '0',
                                     'CLK_DOMAIN': 'design_1_processing_system7_0_0_FCLK_CLK0',
                                     'C_S00_AXI_ADDR_WIDTH': '5',
                                     'C_S00_AXI_BASEADDR': '0x43C00000',
                                     'C_S00_AXI_DATA_WIDTH': '32',
    

In [28]:
j5 = ol.disp_interface_0  #find the AXI MMIO module which we can talk to (name of IP)
#Now it is time to interface with the j5 IP:
# registers are four bytes in size, but address space is byte addressable! Keep in mind!
j5.write(0x08,0)  #write 0 to address location 0x08 (command type)..
j5.write(0x0C,5) #write "5" to address location 0x0C (should show up on green LEDs due to slice
j5.write(0x10,2) #write 2 to address location 0x10
d = j5.read(0x04)  # should read the value of all four push buttons (for test)
print(d)    #print output (hopefully buttons)
d = j5.read(0x00)  # read deadbeef hopefully (hard-coded in your mmio)
print(hex(d))

0
0xdeadbeef


In [29]:
j5.write(0x08,0)  #write 4 to address location 0x08...which is register 2 (command type)
j5.write(0x0C,0x0F000F00) #display test command
j5.write(0x14,0x0) #trigger (write something to register 5)
j5.write(0x0C,0x0C000C00) #turn off display
j5.write(0x14,0x0) #trigger
j5.write(0x0C,0x0C010C01) #display on command
j5.write(0x14,0x0) #trigger
j5.write(0x0C,0x09000900) # decode mode 0
j5.write(0x14,0x0) #trigger
j5.write(0x0C,0x0B070B07) #scan mode command
j5.write(0x14,0x0) # trigger
j5.write(0x0C,0x0A010A01) #screen intensity set
j5.write(0x14,0x0) #trigger

In [30]:
j5.write(0x08,1)  #write 1 to address location 0x00 (drawtype)
for y in range(1,9):
    j5.write(0x10,y)  #write y to address location 0x08 (row address y indexed)
    j5.write(0x0C,0x00000601)
    j5.write(0x14,0x0) #trigger

In [31]:
j5.write(0x08,1)  #write 1 to address location 0x00 (drawtype)
for y in range(1,9):
    j5.write(0x10,y)  #write y to address location 0x08 (row address y indexed)
    j5.write(0x0C,0xAAAAAAAA)
    j5.write(0x14,0x0) #trigger

In [32]:
j5.write(0x08,1)  #write 1 to address location 0x00 (drawtype)
for y in range(1,9):
    j5.write(0x10,y)  #write y to address location 0x08 (row address y indexed)
    j5.write(0x0C,0x55555555)
    j5.write(0x14,0x0) #trigger

In [33]:
import time

In [39]:
j5.write(0x08,1)  #write 1 to address location 0x00 (drawtype)
flag = 1
while flag == 1:
    for y in range(1,9):
        j5.write(0x10,y)  #write y to address location 0x08 (row address y indexed)
        j5.write(0x0C,0x55555555)
        j5.write(0x14,0x0) #trigger
        time.sleep(1)
        j5.write(0x0C,0xAAAAAAAA)
        j5.write(0x14,0x0) #trigger
        if (j5.read(0x04) == 0x1):
            flag = 0
            break
    

In [36]:
print(j5.read(0x04))

1
